# A Quick Introduction to Optuna

This Jupyter notebook goes through the basic usage of Optuna.

- Install Optuna
- Write a training algorithm that involves hyperparameters
  - Read train/valid data
  - Define and train model
  - Evaluate model
- Use Optuna to tune the hyperparameters (hyperparameter optimization, HPO)
- Visualize HPO

## Install `optuna`

Optuna can be installed via `pip` or `conda`.

In [1]:
!pip install --quiet optuna

     |████████████████████████████████| 307kB 3.9MB/s 
     |████████████████████████████████| 174kB 18.4MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 143kB 17.5MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 112kB 19.5MB/s 


In [2]:
import optuna

optuna.__version__

'2.8.0'

## Optimize Hyperparameters

### Define a simple scikit-learn model

We start with a simple random forest model to classify flowers in the Iris dataset. We define a function called `objective` that encapsulates the whole training process and outputs the accuracy of the model.

In [3]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection

def objective():
    iris = sklearn.datasets.load_iris()  # Prepare the data.
    
    clf = sklearn.ensemble.RandomForestClassifier(    
        n_estimators=5, max_depth=3)  # Define the model.
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()  # Train and evaluate the model.

print('Accuracy: {}'.format(objective()))

Accuracy: 0.9466666666666667


### Optimize hyperparameters of the model

The hyperparameters of the above algorithm are `n_estimators` and `max_depth` for which we can try different values to see if the model accuracy can be improved. The `objective` function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [4]:
import optuna

def objective(trial):
    iris = sklearn.datasets.load_iris()
    
    n_estimators = trial.suggest_int('n_estimators', 2, 20)
    max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))
    
    clf = sklearn.ensemble.RandomForestClassifier(
        n_estimators=n_estimators, max_depth=max_depth)
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-06-20 22:02:28,005] A new study created in memory with name: no-name-94eab2f0-4f88-4c24-a7c6-1dc743e4ae3f
[I 2021-06-20 22:02:28,070] Trial 0 finished with value: 0.9266666666666666 and parameters: {'n_estimators': 14, 'max_depth': 1.1580995707822532}. Best is trial 0 with value: 0.9266666666666666.
[I 2021-06-20 22:02:28,099] Trial 1 finished with value: 0.9533333333333333 and parameters: {'n_estimators': 5, 'max_depth': 10.143521700245973}. Best is trial 1 with value: 0.9533333333333333.
[I 2021-06-20 22:02:28,156] Trial 2 finished with value: 0.7600000000000001 and parameters: {'n_estimators': 13, 'max_depth': 1.4779775117084137}. Best is trial 1 with value: 0.9533333333333333.
[I 2021-06-20 22:02:28,181] Trial 3 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 3, 'max_depth': 26.042441674849826}. Best is trial 1 with value: 0.9533333333333333.
[I 2021-06-20 22:02:28,243] Trial 4 finished with value: 0.96 and parameters: {'n_estimators': 13, 'max_dept

Accuracy: 0.9666666666666667
Best hyperparameters: {'n_estimators': 17, 'max_depth': 31.44353740251283}


It is possible to condition hyperparameters using Python `if` statements. We can for instance include another classifier, a support vector machine, in our HPO and define hyperparameters specific to the random forest model and the support vector machine.

In [5]:
import sklearn.svm

def objective(trial):
    iris = sklearn.datasets.load_iris()

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 2, 20)
        max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-06-20 22:02:34,574] A new study created in memory with name: no-name-839f178a-9ff9-4d1e-8d63-24d4f7eec36c
[I 2021-06-20 22:02:34,612] Trial 0 finished with value: 0.7466666666666667 and parameters: {'classifier': 'RandomForest', 'n_estimators': 7, 'max_depth': 1.6458248300063194}. Best is trial 0 with value: 0.7466666666666667.
[I 2021-06-20 22:02:34,679] Trial 1 finished with value: 0.9666666666666667 and parameters: {'classifier': 'RandomForest', 'n_estimators': 16, 'max_depth': 8.201375692811228}. Best is trial 1 with value: 0.9666666666666667.
[I 2021-06-20 22:02:34,693] Trial 2 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'svc_c': 1.1106127225103179e-05}. Best is trial 1 with value: 0.9666666666666667.
[I 2021-06-20 22:02:34,719] Trial 3 finished with value: 0.6666666666666666 and parameters: {'classifier': 'RandomForest', 'n_estimators': 4, 'max_depth': 1.8527679516349724}. Best is trial 1 with value: 0.9666666666666667.
[I 2021-06-20 22:02:34,732] Tria

Accuracy: 0.9733333333333333
Best hyperparameters: {'classifier': 'RandomForest', 'n_estimators': 17, 'max_depth': 5.232116529278027}


### Plotting the study

Plotting the optimization history of the study.

In [6]:
optuna.visualization.plot_optimization_history(study)

Plotting the accuracies for each hyperparameter for each trial.

In [7]:
optuna.visualization.plot_slice(study)

Plotting the accuracy surface for the hyperparameters involved in the random forest model.

In [8]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])